In [18]:
import SkopeDataReader
from copy import copy
import numpy as np
import pickle

# Converts Skope AttrDict to normal dict
def convert_to_dict(ad):
    d = copy(ad.__dict__)
    for key, val in d.items():
        if type(val) == SkopeDataReader.AttrDict.AttrDict:
            d[key] = convert_to_dict(val)
    return d

In [19]:
wave_file = '../acquire/seqs/skope_meas.pickle'
with open(wave_file, 'rb') as handle:
    pp = pickle.load(handle)
    
wave_data = pp['wave_data']
t_trig = pp['t_trigger']

all_diridx = []
for mm in pp['all_meas']:
    if mm['channels'][2] == 'x':
        diridx = 0
    elif mm['channels'][2] == 'y':
        diridx = 1
    elif mm['channels'][2] == 'z':
        diridx = 2
    all_diridx.append(diridx)
    
all_diridx = np.array(all_diridx)

In [15]:
dataPath = 'F:\\data\\2024_1027_girf3\\2024_1024\\'
scanNr = 9

scan = SkopeDataReader.DataReader(dataPath, scanNr)
sd = convert_to_dict(scan.scanDef)
N_d = int(sd['data']['k']['nrInterleaveSamples'])
dt = sd['data']['k']['dt']
delay = sd['data']['k']['delay']
skope_tt = np.arange(N_d) * dt + delay

In [25]:
N_av = 10
N_ax = 3
N_waves = 24
N_fits = 16
N_polarity = 3

kdata = np.zeros([N_av, N_ax, N_waves, N_polarity, N_fits, N_d], np.float32)
all_grad = np.zeros([N_av, N_ax, N_waves, N_polarity, N_d], np.float32)

print(kdata.shape)

for ii, meas in enumerate(pp['all_meas']):
    i_av = meas['i_av']
    i_polarity = meas['polarity'] + 1
    
    if meas['channels'][2] == 'x':
        i_ax = 0
    elif meas['channels'][2] == 'y':
        i_ax = 1
    elif meas['channels'][2] == 'z':
        i_ax = 2
        
    i_wave = meas['idx']
    if meas['mode'] == 'chirp':
        i_wave += 12
        
    kdata[i_av, i_ax, i_wave, i_polarity] = scan.getData('kspha', dynamics=[ii,]).squeeze().transpose()
    
    tt = t_trig[ii] + skope_tt

    if wave_data[i_ax].size > 0:
        if i_ax == 0:
            gg = np.interp(tt, wave_data[i_ax][0], -wave_data[i_ax][1], 0, 0)
        else:
            gg = np.interp(tt, wave_data[i_ax][0], wave_data[i_ax][1], 0, 0)
    else:
        gg = np.zeros_like(tt)
        
    all_grad[i_av, i_ax, i_wave, i_polarity] = gg

(10, 3, 24, 3, 16, 80000)


In [ ]:
kdata.dtype

dtype('float32')